# GGH数字签名
**密钥生成算法：**  
输入：维度n  
输出：私钥sk，公钥pk
1. 生成1个好基$V$, $V \leftarrow \mathbb{Z}^{n \times n}, \mathcal{H}(V) \approx 1$
2. 生成1个坏基$W$，$\mathcal{H}(W) \approx 1$
3. 私钥sk$\leftarrow V$
4. 公钥pk$\leftarrow W$
5. 返回(sk, pk)
----
**数字签名：**  
输入：私钥sk，明文m  
输出：签名s  
1. 用Babai算法求解s $\leftarrow$ solveCVP(V, m)
2. 返回s
----
**验证签名：**  
输入：签名s，明文m  
输出：接受或拒绝  
1. 如果$||s-m|| < \sqrt{n} \sigma(\mathcal{L})$ 则接受
2. 否则，拒绝

**例：** 用三维格对GGH数字签名进行举例。  
考虑一个三维的格$\mathcal{L} \subset \mathbb{R}^3$，用一组“好”基为$$V = 
\left [ \begin{array}{ccc} 
100 & 0 & 0 \\
0 & 90 & 15 \\
0 & 20 & 95
\end{array} \right ]$$
生成“坏”基的幺模矩阵为$$U = 
\left [ \begin{array}{ccc} 
50 & 11 & 2 \\
-5 & -20 & -7 \\
2 & 3 & 1
\end{array} \right ]$$
签名明文为$$m=\left [ \begin{array}{ccc} 
368 & 465 & 593 
\end{array} \right ]$$

#### 生成密钥 

In [1]:
import numpy as np
import simchain.lbc as lbc
# 私钥
good = np.array([[100, 0, 0], [0, 90, 15], [0, 20, 95]])
good

array([[100,   0,   0],
       [  0,  90,  15],
       [  0,  20,  95]])

2019-04-30 14:15:27,798 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# 幺模矩阵
u = np.array([[50, 11, 2], [-5, -20, -7], [2, 3, 1]])
u

array([[ 50,  11,   2],
       [ -5, -20,  -7],
       [  2,   3,   1]])

In [3]:
# 公钥
bad = np.dot(u, good)
bad

array([[ 5000,  1030,   355],
       [ -500, -1940,  -965],
       [  200,   290,   140]])

In [4]:
# 公钥的Hadamard比
lbc.hadamard(good)

0.9762428656955312

In [5]:
# 私钥的Hadamard比
lbc.hadamard(bad)

0.04147703489649823

In [6]:
# 幺模矩阵行列式
np.linalg.det(u)

1.000000000000136

#### 数字签名

In [ ]:
# 明文向量
m = [368, 465, 593]
x = np.linalg.solve()